In [14]:
import numpy as np
import pandas as pd
from tqdm import tqdm

In [15]:
!ls /data2/yt8m/code/yt8m

ls: cannot access '/data2/yt8m/code/yt8m': No such file or directory


In [3]:
# p = np.load('/data2/yt8m/code/yt8m/probs_rnn3_tr_0919_1943.npy', allow_pickle=True) lb0.5xx

In [52]:
p1 = np.load('/data2/yt8m/code/yt8m/softmax/rnn3_ft_1005_5492_lb746.npy', allow_pickle=True)

In [53]:
p2 = np.load('/data2/yt8m/code/yt8m/softmax/rnn4_ft_1006_5625.npy', allow_pickle=True)
p1[:2]

array([[9.8455113e-01, 2.0337227e-07, 3.5731500e-07, ..., 6.0892003e-06,
        1.9596441e-06, 4.1937537e-07],
       [9.9528128e-01, 5.2664177e-06, 1.0759398e-06, ..., 1.2880225e-06,
        1.9060346e-05, 3.0180809e-06]], dtype=float32)

In [54]:
p = np.average([p1, p2], 0, weights=[0.5, 0.5])

In [16]:
#p = np.load('/data2/yt8m/code/yt8m/softmax/rnn6_mt_1004_5838.npy', allow_pickle=True)
#p = np.load('/data2/yt8m/code/yt8m/softmax/rnn3_mt_1004_5910.npy', allow_pickle=True)
#p = np.load('/data2/yt8m/code/yt8m/softmax/rnn3_ft_1005_5492.npy', allow_pickle=True)
#p = np.load('/data2/yt8m/code/yt8m/softmax/rnn4_ft_1006_5625.npy', allow_pickle=True)
#p = np.load('/data2/yt8m/code/yt8m/softmax/rnn6_ft_1006_5385.npy', allow_pickle=True)
#p = np.load('/mnt/chicm/yt8m/code/yt8m/softmax/transformer_3_1010_3490.npy', allow_pickle=True)
p = np.load('/mnt/chicm/yt8m/code/yt8m/softmax/transformer_6_gate_3934.npy', allow_pickle=True)



In [17]:
type(p)

numpy.ndarray

In [18]:
p[:2]

array([[7.9719907e-01, 5.5543386e-04, 1.4763515e-03, ..., 7.6408577e-07,
        2.7500621e-06, 3.7659797e-06],
       [6.4514536e-01, 2.1934361e-04, 2.2770291e-02, ..., 2.7767795e-08,
        1.1964933e-03, 6.6874813e-06]], dtype=float32)

In [19]:
df_test = pd.read_csv('/mnt/chicm/yt8m/3/frame/test_ids.csv')
df_test.head()

,vid,start_time,seg_id
0,MeVj,0,MeVj:0
1,MeVj,5,MeVj:5
2,MeVj,10,MeVj:10
3,MeVj,15,MeVj:15
4,MeVj,20,MeVj:20


In [20]:
def get_classes():
    df = pd.read_csv('/mnt/chicm/yt8m/3/frame/classes_1001.csv')
    c = df['class'].values.tolist()
    stoi = { c[i]: i for i in range(len(c)) }
    return c, stoi

classes, _ = get_classes()

In [21]:
classes[:2]

['none', '3']

In [22]:
print(df_test.shape, p.shape)

(2038114, 3) (2038114, 1001)


In [10]:
res = []
for i in tqdm(range(1, 1001)):
    idx = np.argsort(p[:,i])[::-1][:100000]
    best_segs = df_test.seg_id.values[idx]
    pred_str = ' '.join(best_segs)
    res.append({
        'Class': classes[i],
        'Segments': pred_str
    })



 30%|███       | 303/1000 [01:49<04:58,  2.34it/s]


 60%|██████    | 605/1000 [03:48<02:16,  2.89it/s]


 91%|█████████ | 907/1000 [05:37<00:34,  2.69it/s]


100%|██████████| 1000/1000 [06:17<00:00,  2.30it/s]

In [11]:
df_sub = pd.DataFrame(res)
df_sub.head()

,Class,Segments
0,3,LXlb:10 LXlb:15 LXlb:45 Rbop:65 1shy:80 LXlb:7...
1,7,JF6j:85 6yo2:50 UyfM:85 s5vM:90 QJPE:65 QJPE:1...
2,8,X2XV:140 Ng7L:80 9UGW:95 X2XV:70 X2XV:65 X2XV:...
3,11,owFx:130 owFx:105 owFx:185 R25k:20 owFx:200 R2...
4,12,un0z:215 hOlg:285 8WZN:205 un0z:240 2Ase:85 hO...


In [12]:
df_sub.to_csv('sub_transformer_6_gate_3934.csv', index=False)

In [13]:
!kaggle competitions submit -c youtube8m-2019 -f sub_transformer_6_gate_3934.csv -m 'submit'

100%|████████████████████████████████████████| 810M/810M [00:36<00:00, 23.1MB/s]
Successfully submitted to The 3rd YouTube-8M Video Understanding Challenge

In [ ]:
!ls -lh